In [ ]:
import os
import io
import pyarrow as pa
import pyarrow.dataset as ds
import duckdb
import polars as pl
from memory_profiler import profile 
from datetime import datetime
%load_ext memory_profiler
%load_ext line_profiler

In [ ]:
# data ingest and sort
trade = ds.dataset("~/data/trade_taq2019.arrow", format = "arrow")
quote = ds.dataset("~/data/quote_taq2019.arrow", format = "arrow")

# construct scanner to database
trade = ds.Scanner.from_dataset(trade)
quote = ds.Scanner.from_dataset(quote)

# connect duckdb to database
con = duckdb.connect()

In [ ]:
def window(trade,con):
    
    # window query
    query = "SELECT * FROM trade WHERE Symbol = 'AAPL' AND Sale_Condition LIKE '%O%'"
    
    result = con.query(query).df()

    return result

%memit m=window(trade,con)
m

In [ ]:
def agg(trade,con):
     
    # aggregated timebucket query 
    query = "SELECT DATE_TRUNC('minute',Time) AS one_min, SUM(Trade_Volume),Symbol FROM trade GROUP BY one_min, Symbol ORDER BY one_min"

    result = con.query(query).df()
    
    return result

%memit m=agg(trade,con)
m

In [ ]:
def new_agg(trade,con):

    #aggregating query
    query = "SELECT Symbol,Exchange,AVG(Trade_Price),AVG(Trade_Volume) FROM trade GROUP BY Symbol,Exchange ORDER BY AVG(Trade_Price) DESC"
    
    result = con.query(query).df()
    
    return result

%memit m=new_agg(trade,con)
m

In [ ]:
def bagg(quote,con):
    query = "SELECT DATE_TRUNC('hour', Time) AS hour, Symbol, AVG(0.5*(Offer_Price+Bid_Price)) AS mid FROM quote GROUP BY Symbol, hour ORDER BY hour"
    
    result = con.query(query).df()
    
    return result

%memit m=bagg(quote,con)
m